In [26]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold

In [2]:
newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])
X = newsgroups.data
y = newsgroups.target

No handlers could be found for logger "sklearn.datasets.twenty_newsgroups"


In [14]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(X)

<1786x28382 sparse matrix of type '<type 'numpy.float64'>'
	with 303138 stored elements in Compressed Sparse Row format>

In [30]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(tfidf, y)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1786, n_folds=5, shuffle=True, random_state=241),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [50]:
ready_grid = gs.grid_scores_
C = 0
sc = 0
for g in ready_grid:
    if g.mean_validation_score > sc:
        C = g.parameters['C']
        sc = g.mean_validation_score

In [51]:
clf = SVC(C=C, kernel='linear', random_state=241)
clf.fit(tfidf, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [148]:
words = vectorizer.get_feature_names()
coef = pd.DataFrame(clf.coef_.data, clf.coef_.indices)
coef = coef.apply(abs)
coef = coef.sort_values(coef2.columns[0], ascending=False)
first_words = coef.head(10)
answer = []
for i in first_words.index.tolist():
    answer.append(str(words[i]))
answer.sort()
print answer

['atheism', 'atheists', 'bible', 'god', 'keith', 'moon', 'religion', 'sci', 'sky', 'space']
